### Cloud Development Kit (CDK)
##### Remove Bootstrap Everywhere

In [ ]:
import botoplus.botoplus as botoplus

qualifier = botoplus.deletebootstrap()

accounts = botoplus.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    session = botoplus.defaults(account['awsaccount'])
    ec2 = session.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        
        print(' - '+region['RegionName'])
            
        try:        

            ecr = 'cdk-'+qualifier+'-container-assets-'+account['awsaccount']+'-'+region['RegionName']

            ecr_client = session.client('ecr', region_name = region['RegionName'])

            paginator = ecr_client.get_paginator('list_images')

            pages = paginator.paginate(
                repositoryName = ecr
            )

            for page in pages:
                for image in page['imageIds']:
                    ecr_client.batch_delete_image(
                        repositoryName = ecr,
                        imageIds = [
                            {
                                'imageDigest': image['imageDigest'],
                                'imageTag': image['imageTag']
                            }
                        ]
                    )

        except:
            pass

        bucket = 'cdk-'+qualifier+'-assets-'+account['awsaccount']+'-'+region['RegionName']

        s3_client = session.client('s3')

        try:

            paginator = s3_client.get_paginator('list_objects_v2')

            pages = paginator.paginate(
                Bucket = bucket
            )

            for page in pages:
                for object in page['Contents']:
                    s3_client.delete_object(
                        Bucket = bucket,
                        Key = object['Key']
                    )

        except:
            pass

        try:

            paginator = s3_client.get_paginator('list_object_versions')

            pages = paginator.paginate(
                Bucket = bucket
            )

            for page in pages:
                for object in page['Versions']:
                    s3_client.delete_object(
                        Bucket = bucket,
                        Key = object['Key'],
                        VersionId = object['VersionId']
                    )

            for page in pages:
                for object in page['DeleteMarkers']:
                    s3_client.delete_object(
                        Bucket = bucket,
                        Key = object['Key'],
                        VersionId = object['VersionId']
                    )

        except:
            pass

        try:
            s3_client.delete_bucket(
                Bucket = bucket
            )
        except:
            pass

        try:

            cfn = 'cdk-bootstrap-'+qualifier+'-'+account['awsaccount']+'-'+region['RegionName']

            cfn_client = session.client('cloudformation', region_name = region['RegionName'])

            cfn_client.update_termination_protection(
                EnableTerminationProtection = False,
                StackName = cfn
            )

            cfn_client.delete_stack(
                StackName = cfn
            )

        except:
            pass